In [1]:
import math
import random

import numpy as np
from tqdm import tqdm

from utils import *
from Reed_Solomon import *
from Convolutional import *

### Encoder/Decoders

Create Reed-Solomon Encoder

In [2]:
m = 8
t = 16
# default mode is np array
RS_255_223 = ReedSolomon(m, t)

In [3]:
n = RS_255_223.n
k = RS_255_223.k

n, k

(255, 223)

Create Convolutional Encoder

In [4]:
kern_7_2 = np.array([binary_str_to_array('1111001'), binary_str_to_array('1011011')])
conv = ConvCode(kern_7_2)

### Utility functions
the error is modelled by BSC
the coding scheme is created as concatenation between (255, 223) R-S code and (7, 1/2) convolution code

In [5]:
def simulate_error(ay, p):
    l = ay.shape[0]
    errs = np.random.binomial(1, p, (l,))
    return (ay + errs) % 2

In [6]:
def voyager_encode(msg):
    return conv.encode(RS_255_223.encode(msg))

def voyager_decode(code):
    err_corrected, recovered = RS_255_223.decode(conv.decode(code))
    return recovered

### Initial Trial
Try encoding and decoding using both coding systems

In [7]:
message_length = m * k

In [8]:
msg = generate_binary_array(message_length)

In [9]:
encoded = voyager_encode(msg)

In [10]:
received = simulate_error(encoded, 0.005)

In [11]:
# the hamming distance, how many errors are there?
hamming_dist(encoded, received)

16

In [12]:
recovered = voyager_decode(received)

In [13]:
np.sum(recovered == msg)

1784

In [14]:
np.all(recovered == msg)

True

### Actual experiment

In [15]:
# number of trials
num_trials = 100

# min and max channel error
min_p = 0.0
max_p = 0.15

# number of intervals
num_interval = 60

p_values = np.linspace(min_p, max_p, num_interval+1)

In [16]:
p_values

array([0.    , 0.0025, 0.005 , 0.0075, 0.01  , 0.0125, 0.015 , 0.0175,
       0.02  , 0.0225, 0.025 , 0.0275, 0.03  , 0.0325, 0.035 , 0.0375,
       0.04  , 0.0425, 0.045 , 0.0475, 0.05  , 0.0525, 0.055 , 0.0575,
       0.06  , 0.0625, 0.065 , 0.0675, 0.07  , 0.0725, 0.075 , 0.0775,
       0.08  , 0.0825, 0.085 , 0.0875, 0.09  , 0.0925, 0.095 , 0.0975,
       0.1   , 0.1025, 0.105 , 0.1075, 0.11  , 0.1125, 0.115 , 0.1175,
       0.12  , 0.1225, 0.125 , 0.1275, 0.13  , 0.1325, 0.135 , 0.1375,
       0.14  , 0.1425, 0.145 , 0.1475, 0.15  ])

In [17]:
num_errors_list_list = []
num_failure_list = []

In [18]:
for p in tqdm(p_values):
    num_errors_list = []
    num_failure = 0
    
    # generate num_trials random strings and encode and decode them
    for _ in range(num_trials):
        try:
            # generate random message
            msg = generate_binary_array(message_length)
            # encode, simulate error, decode
            encoded = voyager_encode(msg)
            received = simulate_error(encoded, p)
            recovered = voyager_decode(received)

            num_errors = np.sum(recovered != msg)
            num_errors_list.append(num_errors)
        except:
            # if an exception is thrown, meaning the decode completely failed
            # e.g. lambd.formal_derivative()(e) == alpha^(-inf) i.e. additive identity in GF(2^m)
            num_errors_list.append(message_length)
            num_failure += 1

    num_errors_list_list.append(num_errors_list)
    num_failure_list.append(num_failure)

100%|████████████████████████████████████████████████████████████████████████████| 61/61 [2:28:06<00:00, 145.68s/it]


In [19]:
sim_result = np.array(num_errors_list_list)

In [20]:
np.save('sim_result_b.npy', sim_result)

In [21]:
print(num_failure_list)

[0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 2, 0, 0, 1, 0, 1, 1, 3, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 2, 2, 1, 1]


In [22]:
np.save('sim_result_b_failure.npy', num_failure_list)